### Name: Jaikishan Mohanty, Roll No: 2K22/SWE/07

## Coding Assignment: Implementation and Optimization of GPT-2 Model

- Task 1: Model Implementation and Checkpoints
- Task 2: Architectural changes
- Task 3: Distributed Training

In [1]:
!pip install einops
!pip install rotary-embedding-torch
!pip install torch
!pip install axial-positional-embedding
!pip install transformers

In [2]:
import torch
import torch.nn as nn
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import einops

### Task 1: GPT-2 Model & Checkpoints

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class MultiHeadAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(MultiHeadAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads

        assert (
            self.head_dim * heads == embed_size
        ), "Embedding size needs to be divisible by heads"

        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.fc_out = nn.Linear(heads * self.head_dim, embed_size)

    def forward(self, values, keys, query, mask):
        N = query.shape[0]

        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]
        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        queries = query.reshape(N, query_len, self.heads, self.head_dim)

        values = self.values(values)
        keys = self.keys(keys)
        queries = self.queries(queries)

        energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])

        if mask is not None:
            energy = energy.masked_fill(mask == 0, float("-1e20"))

        attention = torch.nn.functional.softmax(energy / (self.embed_size ** (1 / 2)), dim=3)

        out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(
            N, query_len, self.heads * self.head_dim
        )

        out = self.fc_out(out)
        return out


class GPT2Small(nn.Module):
    def __init__(self, vocab_size, embed_size, heads, depth):
        super(GPT2Small, self).__init__()

        self.embed_size = embed_size
        self.word_embedding = nn.Embedding(vocab_size, embed_size)
        self.positional_embedding = nn.Embedding(1000, embed_size)
        self.layers = nn.ModuleList()

        for _ in range(depth):
            self.layers.append(
                nn.ModuleList(
                    [
                        MultiHeadAttention(embed_size, heads),
                        nn.LayerNorm(embed_size),
                        nn.Linear(embed_size, 4 * embed_size),
                        nn.ReLU(),
                        nn.Linear(4 * embed_size, embed_size),
                    ]
                )
            )

    def forward(self, x, mask):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(device)
        out = self.word_embedding(x) + self.positional_embedding(positions)

        for layer in self.layers:
            attention = layer[0](out, out, out, mask)
            out = out + attention
            out = layer[1](out)
            out = layer[4](layer[3](layer[2](out)))

        return out



vocab_size = 10000
embed_size = 128
heads = 4
depth = 3
model = GPT2Small(vocab_size, embed_size, heads, depth)

input_sequence = torch.randint(0, vocab_size, (32, 20))
mask = (input_sequence != 0).unsqueeze(1).unsqueeze(2)
output = model(input_sequence, mask)
print(output.shape)


torch.Size([32, 20, 128])


### Task 2: Transformer Architectural Changes

In [4]:

model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

input_text = "Once upon a time, in a"
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

output_ids = model.generate(
    input_ids,
    max_length=50,
    num_beams=5,
    no_repeat_ngram_size=2,
    top_k=50,
    top_p=0.95,
    temperature=0.7,
    pad_token_id=tokenizer.eos_token_id,
    attention_mask=torch.ones_like(input_ids)
)
generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print("Generated Text:\n", generated_text)


C:\Users\Jaikishan Mohanty\anaconda3\lib\site-packages\transformers\generation\configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
C:\Users\Jaikishan Mohanty\anaconda3\lib\site-packages\transformers\generation\configuration_utils.py:394: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Generated Text:
 Once upon a time, in a way, I felt like I was in the middle of something.

"It was like, 'Oh my God, this is going to be great.' And then I realized that I wasn't going anywhere.


### Task 3: Training Loop Implementation 

In [5]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

def test_gpt2_small(model, tokenizer, input_text, max_length=50, num_beams=5, temperature=0.7):
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    output_ids = model.generate(
        input_ids,
        max_length=max_length,
        num_beams=num_beams,
        temperature=temperature,
        pad_token_id=tokenizer.eos_token_id,
        attention_mask=torch.ones_like(input_ids)
    )
    generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return generated_text

model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
gpt2_model = GPT2LMHeadModel.from_pretrained(model_name)

input_prompt = "Once upon a time, in a"
output_text = test_gpt2_small(gpt2_model, tokenizer, input_prompt)
print("Test Case 1:\n", "Input Prompt:", input_prompt, "\nGenerated Text:", output_text)
print("="*50)

input_prompt = "In a galaxy far, far away,"
output_text = test_gpt2_small(gpt2_model, tokenizer, input_prompt)
print("Test Case 2:\n", "Input Prompt:", input_prompt, "\nGenerated Text:", output_text)
print("="*50)

input_prompt = "To be or not to be, that is"
output_text = test_gpt2_small(gpt2_model, tokenizer, input_prompt)
print("Test Case 3:\n", "Input Prompt:", input_prompt, "\nGenerated Text:", output_text)


C:\Users\Jaikishan Mohanty\anaconda3\lib\site-packages\transformers\generation\configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Test Case 1:
 Input Prompt: Once upon a time, in a 
Generated Text: Once upon a time, in a time of war, in a time of peace, in a time of peace, in a time of peace, in a time of peace, in a time of peace, in a time of peace, in a time
Test Case 2:
 Input Prompt: In a galaxy far, far away, 
Generated Text: In a galaxy far, far away, there is a galaxy far, far, far away.

In a galaxy far, far away, there is a galaxy far, far, far, far, far, far, far, far, far
Test Case 3:
 Input Prompt: To be or not to be, that is 
Generated Text: To be or not to be, that is not the point.

The point is that if you want to be or not to be, that is not the point.

The point is that if you want to be or not to be
